In [0]:
#Install prerequisite packages
!pip install python-dotenv==1.0.0

!pip install llama-index==0.10.59
!pip install llama-index-llms-openai==0.1.27
!pip install llama-index-embeddings-openai==0.1.11
!pip install llama-index-llms-azure-openai==0.1.10
!pip install llama-index-embeddings-azure-openai==0.1.11


  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Using cached llama_index-0.10.59-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.2.9-py3-none-any.whl.metadata (729 bytes)
  Using cached llama_index_cli-0.1.13-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.10.59-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_openai-0.1.11-py3-none-any.whl.metadata (655 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.6.11-py3-none-any.whl.metadata (3.6 kB)
  Using cached llama_index_legacy-0.9.48.post4-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.1.31-py3-none-any.whl.metadata (650 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.1.9-py3-none-any.whl.metadata (728 bytes)
 

In [0]:
#Setup Azure Open AI connection
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.core import Settings
import os
import nest_asyncio

nest_asyncio.apply()

#API info. Replace with your own keys and end points
api_key=""
azure_endpoint=""
azure_deployment="GPT4-O"
api_version="2024-12-01-preview"

#Setup the LLM
Settings.llm=AzureOpenAI(
    model="gpt-4o",
    deployment_name=azure_deployment,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

#Setup the embedding model RAG
Settings.embed_model= AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key='',
    azure_endpoint='',
    api_version='2023-05-15',
)

In [0]:
from typing import List
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

#-------------------------------------------------------------
# Tool 1 : Function that returns the list of items in an order
#-------------------------------------------------------------
def get_order_items(order_id: int) -> List[str] :
    """Given an order Id, this function returns the 
    list of items purchased for that order"""
    
    order_items = {
            1001: ["Laptop","Mouse"],
            1002: ["Keyboard","HDMI Cable"],
            1003: ["Laptop","Keyboard"]
        }
    if order_id in order_items.keys():
        return order_items[order_id]
    else:
        return []

#-------------------------------------------------------------
# Tool 2 : Function that returns the delivery date for an order
#-------------------------------------------------------------
def get_delivery_date(order_id: int) -> str:
    """Given an order Id, this function returns the 
    delivery date for that order"""

    delivery_dates = {
            1001: "10-Jun",
            1002: "12-Jun",
            1003: "08-Jun"       
    }
    if order_id in delivery_dates.keys():
        return delivery_dates[order_id]
    else:
        return []

#----------------------------------------------------------------
# Tool 3 : Function that returns maximum return days for an item
#----------------------------------------------------------------
def get_item_return_days(item: str) -> int :
    """Given an Item, this function returns the return support
    for that order. The return support is in number of days"""
    
    item_returns = {
            "Laptop"     : 30,
            "Mouse"      : 15,
            "Keyboard"   : 15,
            "HDMI Cable" : 5
    }
    if item in item_returns.keys():
        return item_returns[item]
    else:
        #Default
        return 45

#-------------------------------------------------------------
# Tool 4 : Vector DB that contains customer support contacts
#-------------------------------------------------------------
#Setup vector index for return policies
support_docs=SimpleDirectoryReader(input_files=["/Workspace/Users/yfatima1@optumcloud.com/Customer Service.pdf"]).load_data()

splitter=SentenceSplitter(chunk_size=1024)
support_nodes=splitter.get_nodes_from_documents(support_docs)
support_index=VectorStoreIndex(support_nodes)
support_query_engine = support_index.as_query_engine()


[Trace(request_id=tr-710c90cf1c3f4514b23cd8ad57991953), Trace(request_id=tr-97620c7360604d81952bbbae00c738c4)]

In [0]:
from llama_index.core.tools import FunctionTool

#Create tools for the 3 functions and 1 index
order_item_tool = FunctionTool.from_defaults(fn=get_order_items)
delivery_date_tool = FunctionTool.from_defaults(fn=get_delivery_date)
return_policy_tool = FunctionTool.from_defaults(fn=get_item_return_days)

support_tool = QueryEngineTool.from_defaults(
    query_engine=support_query_engine,
    description=(
        "Customer support policies and contact information"
    ),
)

In [0]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI

#Setup the Agent worker in LlamaIndex with all the Tools
#This is the tool executor process
agent_worker = FunctionCallingAgentWorker.from_tools(
    [order_item_tool, 
     delivery_date_tool,
     return_policy_tool,
     support_tool
    ], 
    llm=Settings.llm, 
    verbose=True
)
#Create an Agent Orchestrator with LlamaIndex
agent = AgentRunner(agent_worker)

In [0]:
#Get return policy for an order
response = agent.query(
    "What is the return policy for order number 1001"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1001
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1001}
=== Function Output ===
['Laptop', 'Mouse']
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1001}
=== Function Output ===
10-Jun


2025/05/23 20:20:48 WARNING mlflow.openai._openai_autolog: Encountered unexpected error when ending trace: 2 validation errors for ChatMessage
tool_calls.0
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...tems'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type
tool_calls.1
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...date'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type


=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Laptop"}
=== Function Output ===
30
=== Calling Function ===
Calling function: get_item_return_days with args: {"item": "Mouse"}
=== Function Output ===
15


2025/05/23 20:20:49 WARNING mlflow.openai._openai_autolog: Encountered unexpected error when ending trace: 2 validation errors for ChatMessage
tool_calls.0
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...tems'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type
tool_calls.1
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...date'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type


=== LLM Response ===
For order number 1001, the items purchased are a Laptop and a Mouse. The return policy for these items is as follows:

- **Laptop**: You can return the laptop within 30 days from the delivery date.
- **Mouse**: You can return the mouse within 15 days from the delivery date.

The delivery date for this order was June 10th.

 Final output : 
 For order number 1001, the items purchased are a Laptop and a Mouse. The return policy for these items is as follows:

- **Laptop**: You can return the laptop within 30 days from the delivery date.
- **Mouse**: You can return the mouse within 15 days from the delivery date.

The delivery date for this order was June 10th.


Trace(request_id=tr-bc1810ebbc864c16a4c14e66fe5196ff)

In [0]:
#Question about an invalid order number
response = agent.query(
    "What is the return policy for order number 1004"
)

print("\n Final output : \n", response)

Added user message to memory: What is the return policy for order number 1004
=== Calling Function ===
Calling function: get_order_items with args: {"order_id": 1004}
=== Function Output ===
[]
=== Calling Function ===
Calling function: get_delivery_date with args: {"order_id": 1004}
=== Function Output ===
[]


2025/05/23 20:21:46 WARNING mlflow.openai._openai_autolog: Encountered unexpected error when ending trace: 2 validation errors for ChatMessage
tool_calls.0
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...tems'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type
tool_calls.1
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...date'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type


=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "return policy"}
=== Function Output ===
The context does not provide information regarding the return policy.


2025/05/23 20:21:47 WARNING mlflow.openai._openai_autolog: Encountered unexpected error when ending trace: 2 validation errors for ChatMessage
tool_calls.0
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...tems'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type
tool_calls.1
  Input should be a valid dictionary or instance of ToolCall [type=model_type, input_value=ChatCompletionMessageTool...date'), type='function'), input_type=ChatCompletionMessageToolCall]
    For further information visit https://errors.pydantic.dev/2.8/v/model_type


=== LLM Response ===
It seems that there is no specific information available for order number 1004, and the general return policy details are not provided either. You may want to contact customer support for more information on return policies.

 Final output : 
 It seems that there is no specific information available for order number 1004, and the general return policy details are not provided either. You may want to contact customer support for more information on return policies.


Trace(request_id=tr-b59bb30bb7d94ec193656a73e8a7dde9)